In [1]:
import os
import json
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras import layers, optimizers, regularizers, callbacks
from tensorflow.keras import Sequential

In [183]:
image_size = 32

In [184]:
import numpy as np
from os import walk
from PIL import Image 

train_data = []
train_labels = []
x = []
for (dirpath, dirnames, filenames) in walk('train/NORMAL/'):
    train_data.extend([filenames])
    for i in range(len(filenames)):
        img = Image.open('train/NORMAL/' + filenames[i])
        img = img.resize((image_size, image_size), Image.ANTIALIAS)
        img = img.convert(mode='RGB')
        
        x.append(img)
        train_labels.extend([0])
    break
for (dirpath, dirnames, filenames) in walk('train/PNEUMONIA//'):
    train_data.extend(filenames)
    for i in range(len(filenames)):
        img = Image.open('train/PNEUMONIA//' + filenames[i])
        img = img.resize((image_size, image_size), Image.ANTIALIAS)
        img = img.convert(mode='RGB')
        
        x.append(img)
        train_labels.extend([1])

    break

In [185]:
#np.array(x[1]).shape
numpy_images = []
for i in range(len(x)):
    numpy_images.append(np.array(x[i]))

In [186]:
# At this point, numpy images are (32,32,3) numpy arrays and the train_labels are all of the respective labels

In [187]:
test_data = []
test_labels = []
y = []
for (dirpath, dirnames, filenames) in walk('test/NORMAL/'):
    test_data.extend([filenames])
    for i in range(len(filenames)):
        img = Image.open('test/NORMAL/' + filenames[i])
        img = img.resize((image_size, image_size), Image.ANTIALIAS)
        img = img.convert(mode='RGB')
        
        y.append(img)
        test_labels.extend([0])
    break
for (dirpath, dirnames, filenames) in walk('test/PNEUMONIA//'):
    test_data.extend(filenames)
    for i in range(len(filenames)):
        img = Image.open('test/PNEUMONIA//' + filenames[i])
        img = img.resize((image_size, image_size), Image.ANTIALIAS)
        img = img.convert(mode='RGB')
        
        y.append(img)
        test_labels.extend([1])
    break

In [188]:
numpy_test_images = []
for i in range(len(y)):
    numpy_test_images.append(np.array(y[i]))


In [189]:
### AT THIS POINT: numpy_test_images[1].shape is made up of (32,32,3) test images and test_labels[150:250] has the labels

In [190]:
train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)

In [191]:
numpy_images = np.asarray(numpy_images)
numpy_test_images = np.asarray(numpy_test_images)

In [192]:
indices = np.arange(numpy_images.shape[0])
np.random.shuffle(indices)

numpy_images =numpy_images[indices]
train_labels = train_labels[indices]


indices = np.arange(numpy_test_images.shape[0])
np.random.shuffle(indices)

numpy_test_images = numpy_test_images[indices]
test_labels = test_labels[indices]

In [193]:
#final_x = np.concatenate(numpy_images, numpy_test_images)
#final_y = np.concatenate(train_labels, test_labels)

## Preprocessing

In [194]:
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.utils import to_categorical

# We normalize the input according to the methods used in the paper
X_train = preprocess_input(numpy_images)
print(len(X_train))
y_train = to_categorical(train_labels)
print(len(y_train))

# We one-hot-encode the labels for training
X_test = preprocess_input(numpy_test_images)
print(len(X_test))
y_test = to_categorical(test_labels)
print(len(y_test))

5216
5216
624
624


## Load and Compile the Model

In [195]:
from tensorflow.keras.applications.densenet import DenseNet121

model = DenseNet121(
    weights=None, 
    include_top=True,
    input_shape=(image_size,image_size,3),
    classes=2
)

# Expand this cell for the model summary
model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_16 (ZeroPadding2 (None, 38, 38, 3)    0           input_9[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 16, 16, 64)   9408        zero_padding2d_16[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1/conv[0][0]                 
________________________________________________________________________________________

__________________________________________________________________________________________________
conv4_block5_1_conv (Conv2D)    (None, 2, 2, 128)    49152       conv4_block5_0_relu[0][0]        
__________________________________________________________________________________________________
conv4_block5_1_bn (BatchNormali (None, 2, 2, 128)    512         conv4_block5_1_conv[0][0]        
__________________________________________________________________________________________________
conv4_block5_1_relu (Activation (None, 2, 2, 128)    0           conv4_block5_1_bn[0][0]          
__________________________________________________________________________________________________
conv4_block5_2_conv (Conv2D)    (None, 2, 2, 32)     36864       conv4_block5_1_relu[0][0]        
__________________________________________________________________________________________________
conv4_block5_concat (Concatenat (None, 2, 2, 416)    0           conv4_block4_concat[0][0]        
          

conv5_block6_0_bn (BatchNormali (None, 1, 1, 672)    2688        conv5_block5_concat[0][0]        
__________________________________________________________________________________________________
conv5_block6_0_relu (Activation (None, 1, 1, 672)    0           conv5_block6_0_bn[0][0]          
__________________________________________________________________________________________________
conv5_block6_1_conv (Conv2D)    (None, 1, 1, 128)    86016       conv5_block6_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_block6_1_bn (BatchNormali (None, 1, 1, 128)    512         conv5_block6_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block6_1_relu (Activation (None, 1, 1, 128)    0           conv5_block6_1_bn[0][0]          
__________________________________________________________________________________________________
conv5_bloc

In [196]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
    shuffle=True
)

model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_16 (ZeroPadding2 (None, 38, 38, 3)    0           input_9[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 16, 16, 64)   9408        zero_padding2d_16[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1/conv[0][0]                 
________________________________________________________________________________________

conv3_block11_1_conv (Conv2D)   (None, 4, 4, 128)    57344       conv3_block11_0_relu[0][0]       
__________________________________________________________________________________________________
conv3_block11_1_bn (BatchNormal (None, 4, 4, 128)    512         conv3_block11_1_conv[0][0]       
__________________________________________________________________________________________________
conv3_block11_1_relu (Activatio (None, 4, 4, 128)    0           conv3_block11_1_bn[0][0]         
__________________________________________________________________________________________________
conv3_block11_2_conv (Conv2D)   (None, 4, 4, 32)     36864       conv3_block11_1_relu[0][0]       
__________________________________________________________________________________________________
conv3_block11_concat (Concatena (None, 4, 4, 480)    0           conv3_block10_concat[0][0]       
                                                                 conv3_block11_2_conv[0][0]       
__________

conv4_block16_1_bn (BatchNormal (None, 2, 2, 128)    512         conv4_block16_1_conv[0][0]       
__________________________________________________________________________________________________
conv4_block16_1_relu (Activatio (None, 2, 2, 128)    0           conv4_block16_1_bn[0][0]         
__________________________________________________________________________________________________
conv4_block16_2_conv (Conv2D)   (None, 2, 2, 32)     36864       conv4_block16_1_relu[0][0]       
__________________________________________________________________________________________________
conv4_block16_concat (Concatena (None, 2, 2, 768)    0           conv4_block15_concat[0][0]       
                                                                 conv4_block16_2_conv[0][0]       
__________________________________________________________________________________________________
conv4_block17_0_bn (BatchNormal (None, 2, 2, 768)    3072        conv4_block16_concat[0][0]       
__________

conv5_block8_0_bn (BatchNormali (None, 1, 1, 736)    2944        conv5_block7_concat[0][0]        
__________________________________________________________________________________________________
conv5_block8_0_relu (Activation (None, 1, 1, 736)    0           conv5_block8_0_bn[0][0]          
__________________________________________________________________________________________________
conv5_block8_1_conv (Conv2D)    (None, 1, 1, 128)    94208       conv5_block8_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_block8_1_bn (BatchNormali (None, 1, 1, 128)    512         conv5_block8_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block8_1_relu (Activation (None, 1, 1, 128)    0           conv5_block8_1_bn[0][0]          
__________________________________________________________________________________________________
conv5_bloc

## Training the model

In [197]:
# # Define callbacks
# checkpoint = callbacks.ModelCheckpoint(
#     'my_model_weights.h5', 
#     monitor='val_acc', 
#     verbose=0, 
#     save_best_only=True, 
#     save_weights_only=False,
#     mode='auto'
# )

# Train the model
hist = model.fit(
    x=X_train,
    y=y_train,
    validation_split=0.1,
    batch_size=64,
    epochs=3,
    verbose=1
)

Train on 4694 samples, validate on 522 samples
Epoch 1/3
4032/4694 [========================>.....] - ETA: 1:05 - loss: 0.2590 - accuracy: 0.8994

KeyboardInterrupt: 

In [173]:
prediction = model.predict(X_test)

In [179]:
correct = 0
total = 0
health = 0
unhealthy = 0
for i in range(len(prediction)):
    res = 0
    if prediction[i][1] >= prediction[i][0]:
        res = 1
        unhealthy += 1
    else:
        health += 1
    if res == test_labels[i]:
        correct += 1
    total += 1
        

In [180]:
print(correct)
print(total)
print(correct/total)
print(health)
print(unhealthy)

390
624
0.625
0
624


## Evaluate and save the Model

In [172]:
model.save_weights('my_model_weights_100px_1epoch.h5')

In [176]:
model.load_weights('my_model_weights_100px_1epoch.h5')
train_loss, train_score = model.evaluate(X_train, y_train)
test_loss, test_score = model.evaluate(X_test, y_test)
print("Train Loss:", train_loss)
print("Test Loss:", test_loss)
print("Train F1 Score:", train_score)
print("Test F1 Score:", test_score)

2208/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

KeyboardInterrupt: 